In [ ]:
import platform, os, sys, datetime, re, itertools
from os.path import join
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = 8,4

In [ ]:
input_files = sorted(glob('../raw_videos/*.avi'))

info = []
for input_file in input_files:
    d,fn    = os.path.split(input_file)
    fn,ext  = os.path.splitext(fn)
    
    split   = fn.lower().split('_')
    try:
        pop,day,age,group,n,date,time = split[:7]
        extra  = None if len(split)==7 else ','.join(split[7:])
        age    = int(age[:-3])
        n_ind  = int(re.findall('n(\d+)[^\d]*',n)[0])
    except:
        print(fn)
        continue

#     # If a repaired version exists, use that instead.
#     repaired_file = os.path.join(d,'repaired',fn+'-repaired'+ext)
#     input_file = repaired_file if os.path.exists(repaired_file) else input_file
    
#     # If a trial file already exists, skip.
#     if os.path.exists(os.path.join(output_dir,'trial.pik')):
#         continue

    info.append(dict( pop=pop, day=day, age=age, group=group, 
                      n_ind=n_ind, date=date, time=time, extra=extra ))

In [ ]:
trials = pd.DataFrame(info,index=input_files)
grouped_trials = trials.groupby(['pop','age','n_ind'])
count  = pd.DataFrame(grouped_trials['pop'].count().rename('count'))
count = count.unstack(1)
count.columns = count.columns.droplevel()
count[pd.isna(count)] = 0
count = count.astype(int)
print('Number of trials of each type:')
count

In [ ]:
# Treat 43dpf and 71dpf as 42dpf and 70dpf respectively.
count2 = count.copy()
count2[42] += count2[43]
count2[70] += count2[71]
count2 = count2.drop(columns=[43,71])

for pop in trials['pop'].unique():
    count2.loc[pop].T.plot(marker='o')
    plt.xlabel('age (dpf)')
    plt.ylabel('number of trials')
    plt.title(pop)